In [ ]:
# Install libs
!pip install -q google-cloud-bigquery pandas plotly pyarrow

# Imports
from google.colab import auth, files
auth.authenticate_user()  # follow the prompt

from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import os
print("Libraries ready.")


Libraries ready.


In [ ]:
# 1) Authenticate (Colab)
from google.colab import auth
auth.authenticate_user()

# 2) Set your real Google Cloud Project ID (from your Cloud Console top bar)
PROJECT_ID = "multi-stop-delivery-route"   # <-- REPLACE with your actual project id if different

if not PROJECT_ID or PROJECT_ID == "YOUR_GCP_PROJECT_ID":
    raise SystemExit("You must set PROJECT_ID to your Google Cloud project id (go to console.cloud.google.com).")

print("Using PROJECT_ID =", PROJECT_ID)


Using PROJECT_ID = multi-stop-delivery-route


In [ ]:
# Create client bound to the explicit project (this avoids empty/default project issues)
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
print("BigQuery client created for project:", client.project)

# Simple test query
try:
    df_test = client.query("SELECT 1 as ok LIMIT 1").to_dataframe()
    print("Test query succeeded:", df_test)
except Exception as e:
    print("Test query failed. Exception:")
    import traceback, sys
    traceback.print_exc(file=sys.stdout)


BigQuery client created for project: multi-stop-delivery-route
Test query succeeded:    ok
0   1


In [ ]:
# re-run your failing query but capture and print the BigQuery exception details
from google.cloud import bigquery
import traceback, sys

sql = "SELECT 1 AS ok LIMIT 1"

try:
    job = client.query(sql)
    df_test = job.to_dataframe()
    print("Query succeeded:", df_test)
except Exception as exc:
    print("== Exception type:", type(exc))
    print("== Exception repr: ", repr(exc))
    print("== Full traceback below ==")
    traceback.print_exc(file=sys.stdout)


Query succeeded:    ok
0   1


In [ ]:
from google.cloud import bigquery

sql = """
SELECT
  FORMAT_DATE('%Y-%m', SAFE.PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING))) AS month,
  EventRootCode,
  COUNT(1) AS cnt
FROM `gdelt-bq.gdeltv2.events`
GROUP BY month, EventRootCode
LIMIT 1
"""
job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
dry_job = client.query(sql, job_config=job_config)

print("Dry run estimate bytes to be processed: {:,}".format(dry_job.total_bytes_processed))


Dry run estimate bytes to be processed: 10,376,301,066


In [ ]:
sql = """
SELECT
  FORMAT_DATE('%Y-%m', SAFE.PARSE_DATE('%Y%m%d', CAST(SQLDATE AS STRING))) AS month,
  CAST(EventRootCode AS STRING) AS event_root,
  COUNT(1) AS event_count,
  AVG(AvgTone) AS avg_tone
FROM `gdelt-bq.gdeltv2.events`
GROUP BY month, event_root
ORDER BY month, event_count DESC
"""

print("Submitting aggregation query...")
job = client.query(sql)

df = job.result().to_dataframe()   # <-- FIXED, NO progress_bar argument

print("Aggregation complete. Rows returned:", len(df))
df.head()



Submitting aggregation query...
Aggregation complete. Rows returned: 4956


,month,event_root,event_count,avg_tone
0,1920-01,04,172880,-1.697359
1,1920-01,01,101929,-2.877878
2,1920-01,19,71007,-5.021896
3,1920-01,05,65517,-0.748825
4,1920-01,02,56614,-2.430036


In [ ]:
# --- Data exploration
import pandas as pd
from IPython.display import display

# Ensure df exists
try:
    df
except NameError:
    raise RuntimeError("`df` not found. Make sure the BigQuery cell (Cell 5) has run successfully.")

print("Shape:", df.shape)

# Top rows
display(df.head())

# Data types, non-null counts
display(df.info())

# Summary statistics (numeric + categorical)
display(df.describe(include='all'))

# Missing values overview
missing = df.isna().sum().sort_values(ascending=False)
print("\nColumns with missing values:")
display(missing[missing > 0])

# Top value counts for object-like columns (limit to first 5)
for col in df.select_dtypes(include=['object', 'category']).columns[:5]:
    print(f"\nTop values for '{col}':")
    display(df[col].value_counts(dropna=False).head(10))


Shape: (4956, 5)


,month,event_root,event_count,avg_tone,event_root_reduced
0,1920-01-01,04,172880,-1.697359,04
1,1920-01-01,01,101929,-2.877878,01
2,1920-01-01,19,71007,-5.021896,Other
3,1920-01-01,05,65517,-0.748825,05
4,1920-01-01,02,56614,-2.430036,02


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4956 entries, 0 to 4955
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   month               4956 non-null   datetime64[ns]
 1   event_root          4956 non-null   string        
 2   event_count         4956 non-null   Int64         
 3   avg_tone            4956 non-null   float64       
 4   event_root_reduced  4956 non-null   object        
dtypes: Int64(1), datetime64[ns](1), float64(1), object(1), string(1)
memory usage: 198.6+ KB


None

,month,event_root,event_count,avg_tone,event_root_reduced
count,4956,4956,4956.0,4956.000000,4956
unique,NaN,22,<NA>,NaN,7
top,NaN,04,<NA>,NaN,Other
freq,NaN,251,<NA>,NaN,3450
mean,2015-04-28 11:36:27.893462272,NaN,130523.711864,-2.465421,NaN
min,1920-01-01 00:00:00,NaN,1.0,-7.462799,NaN
25%,2010-07-01 00:00:00,NaN,38.0,-3.597861,NaN
50%,2015-12-01 00:00:00,NaN,17517.0,-2.490261,NaN
75%,2020-12-01 00:00:00,NaN,210415.25,-1.021236,NaN
max,2025-11-01 00:00:00,NaN,525981.0,2.843702,NaN



Columns with missing values:


,0



Top values for 'event_root_reduced':


,count
event_root_reduced,
Other,3450
04,251
01,251
05,251
02,251
11,251
03,251


In [ ]:
# --- Data Processing / Feature Engineering

proc_df = clean_df.copy()

# 1) Date feature extraction
if 'month' in proc_df.columns:
    proc_df['month'] = pd.to_datetime(proc_df['month'], errors='coerce')
    proc_df['year'] = proc_df['month'].dt.year
    proc_df['month_num'] = proc_df['month'].dt.month
    proc_df['dayofweek'] = proc_df['month'].dt.dayofweek

# 2) One-hot encode categorical columns with low cardinality
cat_cols = proc_df.select_dtypes(include=['object','category','string']).columns
small_cats = [c for c in cat_cols if proc_df[c].nunique() < 20]

if small_cats:
    proc_df = pd.get_dummies(proc_df, columns=small_cats, drop_first=True)

# 3) Log transform skewed numeric columns
num_cols = proc_df.select_dtypes(include='number').columns
for col in num_cols:
    if (proc_df[col] > 0).all() and proc_df[col].skew() > 1:
        proc_df[col + '_log'] = np.log1p(proc_df[col])

print("Data processing complete. New shape:", proc_df.shape)
proc_df.head()


Data processing complete. New shape: (4956, 8)


,month,event_root,event_count,avg_tone,year,month_num,dayofweek,event_count_log
0,1920-01-01,04,172880,-1.697359,1920,1,3,12.060359
1,1920-01-01,01,101929,-2.877878,1920,1,3,11.532042
2,1920-01-01,19,71007,-5.021896,1920,1,3,11.170548
3,1920-01-01,05,65517,-0.748825,1920,1,3,11.09008
4,1920-01-01,02,56614,-2.430036,1920,1,3,10.944029


In [ ]:
# --- Data cleaning
import numpy as np
import pandas as pd # Ensure pandas is imported for pd.api.types

# Work on a copy so we keep original df if needed
clean_df = df.copy()

# 1) Drop rows with >50% missing values (adjust threshold if you prefer)
thresh = int(0.5 * len(clean_df.columns))
clean_df = clean_df.dropna(thresh=thresh)

# 2) Impute numeric columns with median; categorical with mode
num_cols = clean_df.select_dtypes(include='number').columns
cat_cols = clean_df.select_dtypes(include=['object','category']).columns

for c in num_cols:
    clean_df[c] = clean_df[c].fillna(clean_df[c].median())

for c in cat_cols:
    m = clean_df[c].mode()
    if not m.empty:
        clean_df[c] = clean_df[c].fillna(m.iloc[0])

# 3) Outlier handling: IQR capping (winsorize)
def iqr_bounds(s, factor=1.5):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    return q1 - factor * iqr, q3 + factor * iqr

# Apply capping on first N numeric cols to be safe
for c in num_cols[:20]:
    # Check if the column is a nullable integer type (e.g., Int64)
    if pd.api.types.is_integer_dtype(clean_df[c]):
        # Temporarily convert to float for clipping, then back to nullable integer type
        temp_col = clean_df[c].astype('float64')
        lower, upper = iqr_bounds(temp_col)
        # For count-like data, ensure the lower bound is not negative
        if c == 'event_count': # Apply specifically for count column
            lower = max(0.0, lower)
        clean_df[c] = temp_col.clip(lower, upper).round().astype(clean_df[c].dtype) # Added .round() here
    else:
        # For other numeric types (e.g., float64), clip directly
        lower, upper = iqr_bounds(clean_df[c])
        clean_df[c] = clean_df[c].clip(lower, upper)

# 4) Trim string columns
for c in cat_cols:
    clean_df[c] = clean_df[c].astype("string").str.strip()

print("Cleaning finished. New shape:", clean_df.shape)
display(clean_df.head())


Cleaning finished. New shape: (4956, 4)


,month,event_root,event_count,avg_tone
0,1920-01,04,172880,-1.697359
1,1920-01,01,101929,-2.877878
2,1920-01,19,71007,-5.021896
3,1920-01,05,65517,-0.748825
4,1920-01,02,56614,-2.430036


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
# --- Data integration
# If you already have a second table/query, name it `df2` or define `sql_2` above to load it.
try:
    df2
except NameError:
    df2 = None
    if "sql_2" in globals():
        try:
            from google.cloud import bigquery
            client = bigquery.Client()
            df2 = client.query(sql_2).to_dataframe()
            print("Loaded df2 from BigQuery using sql_2.")
        except Exception as e:
            print("Failed to load df2 from sql_2:", e)

if df2 is None:
    print("No df2 detected. If you want to integrate another table, define `df2` or `sql_2` and re-run this cell.")
    proc_base = clean_df.copy()
else:
    # Find candidate join keys (common column names)
    common = set(clean_df.columns).intersection(set(df2.columns))
    if not common:
        raise RuntimeError("No common columns found between datasets for joining. Consider renaming or creating keys.")
    # Choose a key (if multiple, pick the first — adjust as required)
    key = list(common)[0]
    print("Joining clean_df with df2 on key:", key)
    proc_base = clean_df.merge(df2, how='left', on=key, suffixes=('','_df2'))
    print("Merged shape:", proc_base.shape)

# `proc_base` is the dataframe to feed into subsequent processing/plotting
display(proc_base.head())
df = proc_base.copy()


No df2 detected. If you want to integrate another table, define `df2` or `sql_2` and re-run this cell.


,month,event_root,event_count,avg_tone
0,1920-01,04,172880,-1.697359
1,1920-01,01,101929,-2.877878
2,1920-01,19,71007,-5.021896
3,1920-01,05,65517,-0.748825
4,1920-01,02,56614,-2.430036


In [ ]:
import pandas as pd

df['month'] = pd.to_datetime(df['month'] + "-01")

TOP_N = 6  # number of categories to highlight
top_roots = df.groupby('event_root')['event_count'].sum().sort_values(ascending=False).head(TOP_N).index.tolist()
print("Top event categories:", top_roots)

df['event_root_reduced'] = df['event_root'].apply(lambda x: x if x in top_roots else 'Other')

agg_plot = df.groupby(['month','event_root_reduced'])['event_count'].sum().reset_index()
agg_plot['month_total'] = agg_plot.groupby('month')['event_count'].transform('sum')
agg_plot['share'] = agg_plot['event_count'] / agg_plot['month_total']

agg_plot.head()


Top event categories: ['04', '01', '05', '02', '03', '11']


,month,event_root_reduced,event_count,month_total,share
0,1920-01-01,01,101929,789562,0.129096
1,1920-01-01,02,56614,789562,0.071703
2,1920-01-01,03,47560,789562,0.060236
3,1920-01-01,04,172880,789562,0.218957
4,1920-01-01,05,65517,789562,0.082979


In [ ]:
import plotly.express as px

fig = px.area(
    agg_plot,
    x='month',
    y='share',
    color='event_root_reduced',
    labels={
        'share': 'Share of Global Events',
        'month': 'Month',
        'event_root_reduced': 'Event Category'
    },
    title='GDELT — Monthly Composition of Global Event Categories (Top 6 + Other)'
)

fig.update_layout(hovermode='x unified')
fig.show()

# Save
fig.write_html("gdelt_eventroot_stack_area.html")
agg_plot.to_csv("gdelt_month_eventroot_agg.csv", index=False)

print("Saved aggregated CSV and HTML visualization.")


Saved aggregated CSV and HTML visualization.


In [ ]:
from google.colab import files
files.download("gdelt_month_eventroot_agg.csv")
files.download("gdelt_eventroot_stack_area.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>